go to https://account.polar.com and download your data

In [10]:
import os
workingDataFile = "/home/chowder/Documents/workingData/polar/hr/polarHRdf.parquet.gzip"

# Get the list of all files and directories
exportPath = "/home/chowder/Documents/dataExports/polar/"
exportfn = "polar-user-data-export_156577f0-84e7-434c-914f-66e092eb137c"
dir_list = os.listdir(exportPath + exportfn)

In [11]:
hrFilenames = [x for x in dir_list if x.split("-")[0] == "training"]
hrFilenames[0]

'training-session-2019-10-27-4001044490-6e165d49-b8b5-4ecf-acb6-0bc1335bf278.json'

In [12]:
import json
data = json.load(open(exportPath + exportfn + "/" + hrFilenames[0]))
data["exercises"][0]["samples"]['heartRate'][0]
# exercises

{'dateTime': '2019-10-27T16:16:57.000', 'value': 95}

In [13]:
import json
import pandas as pd

columnNames = ["sampleDT", "value"]

samplesCount = 0
samplesList = []

for fn in hrFilenames:
    print(fn)
    data = json.load(open(exportPath + exportfn + "/" + fn))

    samples = data["exercises"][0]["samples"]
    if 'heartRate' in samples:
        for row in samples['heartRate']:
            if len(row) == 2:
                samplesList.append([pd.to_datetime(row["dateTime"] + "-0700"), row["value"]])
                samplesCount += 1
                if samplesCount % 20_000 == 0:
                    print(samplesCount)


training-session-2019-10-27-4001044490-6e165d49-b8b5-4ecf-acb6-0bc1335bf278.json
training-session-2020-05-26-4704617257-acb12704-d279-48c3-be87-b71597e8669c.json
20000
40000
60000
training-session-2024-07-16-7921452466-a7324d4b-db7d-4f9e-a408-4027f96be4d1.json
training-session-2024-07-18-7922789379-d653bdf2-ab9d-49b7-92fa-d9aac76f33c4.json
80000
training-session-2020-05-28-4710506157-6e626658-e3f6-43be-ad12-329e6bd20709.json
training-session-2019-10-27-3997220845-24165c43-53a6-4bcc-a104-2e1b696dfae0.json
training-session-2019-10-30-4007105070-2f06cc5b-557c-4958-b7af-c21b5a563229.json
training-session-2024-07-24-7926566566-0858fb7b-8688-4b79-a5dd-24cd9947671c.json
training-session-2019-10-27-4001044540-14a76d51-93eb-496c-88a1-40d6c3a868f5.json
training-session-2019-10-27-3997190235-ef37fe2f-8793-40b9-a81b-e2fdd326973d.json
training-session-2019-10-30-4007108755-608ac9ec-ab9e-4abd-bfdd-95704f62b8de.json
training-session-2024-08-07-7936183748-5763bd67-6c0b-4b81-8fc7-674d6791b06e.json
1000

In [14]:
samplesList = sorted(samplesList, key=lambda x:x[0])
samplesList = [x for x in samplesList if x[1] > 0]

In [15]:
import pandas as pd

polarHRdf = pd.DataFrame(data=samplesList, columns=columnNames)

polarHRdf = polarHRdf.set_index("sampleDT")

In [16]:
polarHRdf

,value
sampleDT,
2019-10-27 15:32:45-07:00,69
2019-10-27 15:32:46-07:00,69
2019-10-27 15:32:47-07:00,69
2019-10-27 15:32:48-07:00,69
2019-10-27 15:32:49-07:00,69
...,...
2024-08-07 22:54:04-07:00,82
2024-08-07 22:54:05-07:00,82
2024-08-07 22:54:06-07:00,83


In [17]:
polarHRdf.dtypes

value    int64
dtype: object

In [18]:
polarHRdf.to_parquet(workingDataFile,
              compression='gzip') 